In [1]:
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import os

In [44]:
load_dotenv()

True

In [10]:
eco = pd.read_csv(os.getenv('CLEANED_DATA_PATH') + '/' + 'eco-products.csv', encoding='ISO-8859-1', sep='\t')
clt = pd.read_csv(os.getenv('CLEANED_DATA_PATH') + '/' + 'TJ38.csv', encoding='ISO-8859-1', sep='\t')
ctr = pd.read_csv(os.getenv('CLEANED_DATA_PATH') + '/' + 'TJ7S.csv', encoding='ISO-8859-1', sep='\t', low_memory=False)

## Get only the physical clients

In [11]:
# theses are the only columns that matters
ctr = ctr[['COCO', 'COMAX', 'COPRO' ]]

In [12]:
ctr = ctr[ctr['COMAX'].isin(clt['COMAX'])]

In [13]:
ctr.shape

(32773662, 3)

## Get the best ecological product

In [33]:
eco = [str(i).zfill(5) for i in eco['COPRO']]

In [37]:
## ECO: 00003 | 242568 - LIVRET DEVELOPPEMENT DURABLE ET SOLIDAIRE
## DEPOTS MONETAIRES - EPARGNE MONETAIRE LIQUIDE - LIVRETS REGLEMENTES
ctr[ctr['COPRO'].isin(eco)]['COPRO'].value_counts()

00003    242568
00546      5141
00548      4785
07649      2724
00940      2166
00509      1876
00547      1027
00941       388
03992       101
00565        83
06458        75
Name: COPRO, dtype: int64

## Concatinate with other tables TJ39 - TJDR - TJER

In [46]:
tj39 = pd.read_csv(os.getenv('DATA_PATH') + '/' + 'tj39.csv', encoding='ISO-8859-1', sep=';')
tjdr = pd.read_csv(os.getenv('DATA_PATH') + '/' + 'tjdr.csv', encoding='ISO-8859-1', sep=';')
tjer = pd.read_csv(os.getenv('DATA_PATH') + '/' + 'tjer.csv', encoding='ISO-8859-1', sep=';')

In [53]:
tj39 = tj39[['COMAX', 'MSMENC', 'MTPATR', 'MTVAOP']]
tjdr = tjdr[['COMAX', 'MTRVIM', 'MTRVFR', 'QTPAFI', 'COHAVI']]
tjer = tjer[['COMAX', 'MTAPJE', 'MTCPJE', 'MTPJE']]

In [66]:
# This table is only good fot the moral clients only
# data = pd.merge(clt,tj39,on='COMAX',how='left')
data = pd.merge(clt,tjer,on='COMAX',how='left')
data = pd.merge(data,tjdr,on='COMAX',how='left')

In [69]:
data = data.dropna()

In [78]:
data = data.drop_duplicates(subset=['COMAX'])

In [79]:
data.shape

(6372, 12)

## les avec le 00003

In [80]:
ids = ctr[ctr['COPRO'] == '00003']['COMAX'].values

In [83]:
data['ECO'] = data['COMAX'].isin(ids)

In [84]:
data.head()

,COMAX,AGE,COSEXE,CTSCPI,MTAPJE,MTCPJE,MTPJE,MTRVIM,MTRVFR,QTPAFI,COHAVI,ECO
67,07a5ef7d08,42,1,3400,0.0,0.0,2500.0,1.0,0.0,0.0,1.0,False
404,bf8d8de347,45,0,6300,0.0,0.0,0.0,0.0,29134.0,3.0,1.0,False
892,0ba988ba1b,65,1,7600,0.0,0.0,0.0,0.0,22950.0,2.0,1.0,True
922,0faa41d303,73,1,8600,0.0,0.0,1600.0,1.0,0.0,0.0,1.0,True
1019,ba1161e851,57,1,5400,0.0,0.0,0.0,57.0,0.0,0.0,1.0,True


## Model

In [88]:
# Labels are the values we want to predict
labels = np.array(data['ECO'])

In [90]:
# Remove the labels from the features
features = np.array(data[data.columns[1:-1]])

In [91]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split

In [92]:
# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.2, random_state = 42)

In [102]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score

In [94]:
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)

# Train the model on training data
rf.fit(train_features, train_labels);

In [113]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)

# Calculate the accurancy score
print(accuracy_score(test_labels, predictions.round()))

0.6180392156862745
